In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download('stopwords')
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  7 08:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket3224' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path + b.name)

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
import builtins

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  tf_list = []
  tf_counter = Counter()

  valid_tokens = (token for token in tokens if token not in all_stopwords)
  tf_counter = Counter(valid_tokens)

  for token in tf_counter.keys():
    tf_list.append((token, (id, tf_counter[token])))

  return tf_list


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl)

  
def calculate_term_total(postings):
    """ Calculates total frequency for each term from the postings RDD. """
    # Using builtins.sum ensures no conflict with Spark SQL
    return postings.mapValues(lambda posting_list: builtins.sum(tf for doc_id, tf in posting_list))

def write_term_total_only(term_total_dict, bucket_name, file_name="term_total.pkl"):
    """ Writes just the term_total dictionary to the bucket using pickle (binary compression). """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f'postings_gcp/{file_name}')
    
    # Write as binary pickle
    with blob.open("wb") as f:
        pickle.dump(term_total_dict, f)
    
    print(f"Successfully saved term_total to gs://{bucket_name}/postings_gcp/{file_name}")



In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)  # create postings as usual
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50) 
print("Postings ready")
w2total_rdd = calculate_term_total(postings_filtered)
print("Calculated Term Total")
w2total_dict = w2total_rdd.collectAsMap()
print("Collected Term Total")
# Save this dictionary to the bucket
write_term_total_only(w2total_dict, bucket_name)

Postings ready
Calculated Term Total


Collected Term Total
Successfully saved term_total to gs://bucket3224/postings_gcp/term_total.pkl
